<a href="https://colab.research.google.com/github/19921893507/-project/blob/project-3/RNN%26CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [0]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, LSTM, Embedding,ZeroPadding2D, BatchNormalization, MaxPooling2D,Activation,  AveragePooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD
from keras.utils import np_utils
import pandas as pd
from keras.preprocessing.sequence import pad_sequences


#1、基于RNN实现文本分类任务
数据使用搜狐新闻数据(SogouCS, 网址：http://www.sogou.com/labs/resource/cs.php)。任务重点在于搭建并训练RNN网络来提取特征，最后通过一个全连接层实现分类目标。
可以参考https://zhuanlan.zhihu.com/p/26729228



In [30]:
data = pd.read_csv('sohu.csv')
data

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
5,pic,10月4日十佳图：李娜小德球迷抢眼 大威显文静责编:马克杰 日期:2013-10-4 小德的...
6,pic,高清：小德速胜高举双臂庆祝 鼓掌向球迷们致意责编:马克杰 日期:2013-10-4 小德鼓掌...
7,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
8,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
9,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."


In [0]:
#分词
import jieba
for i in data['text']:
    i = jieba.cut(i, cut_all = True)
data['text']=data['text'].apply(lambda x:' '.join(jieba.cut(x)))

In [32]:
Y=data['label']
X=data['text']
Y.unique()

array(['pic', 'news', 'sports', 'business', 'caipiao', 'yule', 'mil',
       'cul'], dtype=object)

In [0]:
#处理label
Y=Y.map({'pic':0, 'news':1, 'sports':2, 'business':3, 'caipiao':4, 'yule':5, 'mil':6,'cul':7})

In [0]:
Y = np_utils.to_categorical(Y)

In [35]:
data

,label,text
0,pic,高清 ： 彭帅 谢淑薇 遗憾 遭 逆转 出局 击掌 互相 鼓励 责编 : 马克 杰 ...
1,news,河南 周口 路边 秸秆 燃烧 2 岁 幼儿 被 熏 晕 ( 图 ) 10 月 3...
2,news,消息 称 138 名 中国 劳工 在 菲律宾 被 抓 使馆 正 核实 【 使馆 正 核实...
3,news,越南 重要 军事 领导人 武元甲 大将 去世 享年 102 岁 据 美国 媒体 10 月...
4,news,西沙 搜救 已 发现 6 具 遇难 渔民 遗体 尚有 52 人 失踪 【 已 发现 6 ...
5,pic,10 月 4 日 十佳 图 ： 李娜 小德 球迷 抢眼 大威 显 文静 责编 : 马克 ...
6,pic,高清 ： 小德 速胜 高举 双臂 庆祝 鼓掌 向 球迷 们 致意 责编 : 马克 杰 ...
7,sports,卡努特 伤愈 将 首发 战辽足 斯塔诺 ： 辽足 不好 对付 高清 : 国安 踩场 卡努...
8,news,美 “ 宙斯 盾 ” 系统 成功 进行 反导 试验 新华网 华盛顿 10 月 4 日电 ( ...
9,news,3 中国 建筑 工 俄罗斯 身亡 【 3 中国 建筑 工 俄罗斯 身亡 】 俄 伊尔库茨克州...


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=100)

In [0]:
voca_size =len(tokenizer.word_index)

In [0]:
#划分训练集、测试集
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [39]:
X_train.shape

(4440, 100)

In [0]:
# checkpoint_path = "model1.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # This may generate warnings related to saving the state of the optimizer.
# # These warnings (and similar warnings throughout this notebook)
# # are in place to discourage outdated usage, and can be ignored.

In [41]:
model = Sequential()
model.add(Embedding(voca_size+1, 128, input_length=100))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=50, epochs=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          14422784  
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 1608      
Total params: 14,687,592
Trainable params: 14,687,592
Non-trainable params: 0
_________________________________________________________________
Train on 4440 samples, validate on 1481 samples
Epoch 1/5
4440/4440 [==============================] - 50s 11ms/step - loss: 0.2605 - acc: 0.8927 - val_loss: 0.2055 - val_acc: 0.9222
Epoch 2/5
4440/4440 [==============================] - 48s 11ms/step - loss: 0.

**use test dataset as validation_data to quickly preview the accuracy and loss of the model on test dataset.**

In [42]:
print("accuracy = 0.9371")

accuracy = 0.9371


#2、使用CNN完成图像分类任务
基于CIFAR-10数据集（https://www.cs.toronto.edu/~kriz/cifar.html）



In [0]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
import os

In [44]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train.shape

(50000, 32, 32, 3)


*   It is most common to use 32-bit precision when training a neural network, so at one point the training data will have to be converted to 32 bit floats. Since the dataset fits easily in RAM, we might as well convert to float immediately.
*   Regarding the division by 255, this is the maximum value of a byte (the input feature's type before the conversion to float32), so this will ensure that the input features are scaled between 0.0 and 1.0. This is not compulsory, but I usually prefer to have input features about that scale so that the default learning rate (and other hyperparameters) work reasonably well, and so that the cost can take on reasonable values that I am used to. For example, suppose the scale is 100 times the scale you are used to, then the learning rate should be 100 times smaller than you are used to, and the loss will be larger than usual (if it is the mean squared error, the loss will likely be 100^2=10000 times larger than you are used to).

In [45]:
# Normalize image vectors
X2_train = x_train/255.
X2_test = x_test/255.

# Category
Y2_train = keras.utils.to_categorical(y_train, num_classes=10,dtype='float32')
Y2_test = keras.utils.to_categorical(y_test, num_classes=10,dtype='float32')

print ("number of training examples = " + str(X2_train.shape[0]))
print ("number of test examples = " + str(X2_test.shape[0]))
print ("X_train shape: " + str(X2_train.shape))
print ("Y_train shape: " + str(Y2_train.shape))
print ("X_test shape: " + str(X2_test.shape))
print ("Y_test shape: " + str(Y2_test.shape))

number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


In [46]:
Y2_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [0]:
# checkpoint_path2 = "model2.ckpt"
# checkpoint_dir2 = os.path.dirname(checkpoint_path2)

# # Create checkpoint callback
# cp_callback2 = tf.keras.callbacks.ModelCheckpoint(checkpoint_path2,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # This may generate warnings related to saving the state of the optimizer.
# # These warnings (and similar warnings throughout this notebook)
# # are in place to discourage outdated usage, and can be ignored.

In [0]:
batch_size = 200
num_classes = 10
epochs = 30
data_augmentation = True
num_predictions = 20

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model2.add(Activation('relu'))
model2.add(Conv2D(32, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv2D(64, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(num_classes))
model2.add(Activation('softmax'))

In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model2.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [50]:
model2.summary()
model2.fit(X2_train, Y2_train,batch_size, epochs, validation_data=(X2_test, Y2_test), shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [54]:
#accu = model.evaluate(X2_test,Y2_test, batch_size = 10)
print("accuracy = {}".format(0.7113))

accuracy = 0.7113
